In [1]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

/opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import InequalityToEquality, IntegerToBinary, LinearEqualityToPenalty

In [3]:
qp = QuadraticProgram()
qp.binary_var('x01')
qp.binary_var('x02')
qp.binary_var('x10')
qp.binary_var('x12')
qp.binary_var('x20')
qp.binary_var('x21')
qp.maximize(linear=[4, 8, 0, 8, 0, 4])
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 4 x01 + 8 x02 + 8 x12 + 4 x21
Subject To

Bounds
 0 <= x01 <= 1
 0 <= x02 <= 1
 0 <= x10 <= 1
 0 <= x12 <= 1
 0 <= x20 <= 1
 0 <= x21 <= 1

Binaries
 x01 x02 x10 x12 x20 x21
End



In [4]:
qp.linear_constraint(linear={'x01':1, 'x02':1}, sense='==', rhs=1, name='start')
qp.linear_constraint(linear={'x01':1, 'x21':1}, sense='<=', rhs=1, name='one1')
qp.linear_constraint(linear={'x02':1, 'x12':1}, sense='<=', rhs=1, name='one2')
qp.linear_constraint(linear={'x01':4, 'x02':6,
                             'x10':2, 'x12':5,
                             'x20':4, 'x21':5}, sense='<=', rhs=9, name='deadline')
print(qp.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 4 x01 + 8 x02 + 8 x12 + 4 x21
Subject To
 start: x01 + x02 = 1
 one1: x01 + x21 <= 1
 one2: x02 + x12 <= 1
 deadline: 4 x01 + 6 x02 + 2 x10 + 5 x12 + 4 x20 + 5 x21 <= 9

Bounds
 0 <= x01 <= 1
 0 <= x02 <= 1
 0 <= x10 <= 1
 0 <= x12 <= 1
 0 <= x20 <= 1
 0 <= x21 <= 1

Binaries
 x01 x02 x10 x12 x20 x21
End



In [5]:
ineq2wq = InequalityToEquality()
qp_eq = ineq2wq.convert(qp)
print(qp_eq.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 4 x01 + 8 x02 + 8 x12 + 4 x21
Subject To
 start: x01 + x02 = 1
 one1: x01 + x21 + one1@int_slack = 1
 one2: x02 + x12 + one2@int_slack = 1
 deadline: 4 x01 + 6 x02 + 2 x10 + 5 x12 + 4 x20 + 5 x21 + deadline@int_slack
            = 9

Bounds
 0 <= x01 <= 1
 0 <= x02 <= 1
 0 <= x10 <= 1
 0 <= x12 <= 1
 0 <= x20 <= 1
 0 <= x21 <= 1
       one1@int_slack <= 1
       one2@int_slack <= 1
       deadline@int_slack <= 9

Binaries
 x01 x02 x10 x12 x20 x21

Generals
 one1@int_slack one2@int_slack deadline@int_slack
End



In [6]:
int2bin = IntegerToBinary()
qp_eq_bin = int2bin.convert(qp_eq)
print(qp_eq_bin.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 4 x01 + 8 x02 + 8 x12 + 4 x21
Subject To
 start: x01 + x02 = 1
 one1: x01 + x21 + one1@int_slack@0 = 1
 one2: x02 + x12 + one2@int_slack@0 = 1
 deadline: 4 x01 + 6 x02 + 2 x10 + 5 x12 + 4 x20 + 5 x21 + deadline@int_slack@0
           + 2 deadline@int_slack@1 + 4 deadline@int_slack@2
           + 2 deadline@int_slack@3 = 9

Bounds
 0 <= x01 <= 1
 0 <= x02 <= 1
 0 <= x10 <= 1
 0 <= x12 <= 1
 0 <= x20 <= 1
 0 <= x21 <= 1
 0 <= one1@int_slack@0 <= 1
 0 <= one2@int_slack@0 <= 1
 0 <= deadline@int_slack@0 <= 1
 0 <= deadline@int_slack@1 <= 1
 0 <= deadline@int_slack@2 <= 1
 0 <= deadline@int_slack@3 <= 1

Binaries
 x01 x02 x10 x12 x20 x21 one1@int_slack@0 one2@int_slack@0 deadline@int_slack@0
 deadline@int_slack@1 deadline@int_slack@2 deadline@int_slack@3
End



In [7]:
lineq2penalty = LinearEqualityToPenalty()
qubo = lineq2penalty.convert(qp_eq_bin)
print(qubo.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 1904 x01 + 2808 x02 + 900 x10 + 2308 x12 + 1800 x20 + 2304 x21
      + 50 one1@int_slack@0 + 50 one2@int_slack@0 + 450 deadline@int_slack@0
      + 900 deadline@int_slack@1 + 1800 deadline@int_slack@2
      + 900 deadline@int_slack@3 + [ - 900 x01^2 - 2500 x01*x02 - 800 x01*x10
      - 2000 x01*x12 - 1600 x01*x20 - 2100 x01*x21 - 100 x01*one1@int_slack@0
      - 400 x01*deadline@int_slack@0 - 800 x01*deadline@int_slack@1
      - 1600 x01*deadline@int_slack@2 - 800 x01*deadline@int_slack@3
      - 1900 x02^2 - 1200 x02*x10 - 3100 x02*x12 - 2400 x02*x20 - 3000 x02*x21
      - 100 x02*one2@int_slack@0 - 600 x02*deadline@int_slack@0
      - 1200 x02*deadline@int_slack@1 - 2400 x02*deadline@int_slack@2
      - 1200 x02*deadline@int_slack@3 - 200 x10^2 - 1000 x10*x12 - 800 x10*x20
      - 1000 x10*x21 - 200 x10*deadline@int_slack@0
      - 400 x10*deadline@int_slack@1 - 800 x10*deadline@int_s

In [8]:
op, offset = qubo.to_ising()
print('offset: {}'.format(offset))
print('operator:')
print(op)

offset: 2775.5
operator:
SummedOp([
  -860.5 * IIIIIIIIIIIZ,
  -1283.5 * IIIIIIIIIIZI,
  -1071.0 * IIIIIIIIZIII,
  -1073.0 * IIIIIIZIIIII,
  -12.5 * IIIIIZIIIIII,
  -12.5 * IIIIZIIIIIII,
  -425.0 * IIIIIIIIIZII,
  -850.0 * IIIIIIIZIIII,
  -212.5 * IIIZIIIIIIII,
  -425.0 * IIZIIIIIIIII,
  -850.0 * IZIIIIIIIIII,
  -425.0 * ZIIIIIIIIIII,
  312.5 * IIIIIIIIIIZZ,
  100.0 * IIIIIIIIIZIZ,
  150.0 * IIIIIIIIIZZI,
  250.0 * IIIIIIIIZIIZ,
  387.5 * IIIIIIIIZIZI,
  125.0 * IIIIIIIIZZII,
  200.0 * IIIIIIIZIIIZ,
  300.0 * IIIIIIIZIIZI,
  100.0 * IIIIIIIZIZII,
  250.0 * IIIIIIIZZIII,
  262.5 * IIIIIIZIIIIZ,
  375.0 * IIIIIIZIIIZI,
  125.0 * IIIIIIZIIZII,
  312.5 * IIIIIIZIZIII,
  250.0 * IIIIIIZZIIII,
  12.5 * IIIIIZIIIIIZ,
  12.5 * IIIIIZZIIIII,
  12.5 * IIIIZIIIIIZI,
  12.5 * IIIIZIIIZIII,
  50.0 * IIIZIIIIIIIZ,
  75.0 * IIIZIIIIIIZI,
  25.0 * IIIZIIIIIZII,
  62.5 * IIIZIIIIZIII,
  50.0 * IIIZIIIZIIII,
  62.5 * IIIZIIZIIIII,
  100.0 * IIZIIIIIIIIZ,
  150.0 * IIZIIIIIIIZI,
  50.0 * IIZIIIIIIZII,
  

In [9]:
aqua_globals.random_seed = 10598
quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
exact_mes = NumPyMinimumEigensolver()

In [10]:
qaoa = MinimumEigenOptimizer(qaoa_mes)
exact = MinimumEigenOptimizer(exact_mes)

In [11]:
exact_result = exact.solve(qubo)
print(exact_result)

optimal function value: 12.0
optimal value: [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
status: SUCCESS


In [12]:
qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

optimal function value: 12.0
optimal value: [1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
status: SUCCESS
